# Wurstpresse
## Skript zur Vorbereitung des Bildimports in iDAI.Objects bzw. Arachne
*Version 1.7 - Dezember 2022*

*von Lukas Lammers für's FAIR.rdm im Spp2143 "Entangled Africa"*

Die Codeblöcke sind jeweils der Reihe nach einzeln auszuführen!
Es gibt vier Abschnitte:
Die Vorbereitungsphase muss bei jedem Durchlauf ausgeführt werden.
Der erste Teil füllt automatisch Importtabellen mit den zugehörigen Seriennummern aus und gibt eine sepatate CSV aus, in denen alle Bilder mit Materialangeben oder unbekannte Fundplatzzuordnungen gesammelt sind.
Der zweite Teil benennt Bilder automatisch um und verschiebt sie in Ordnerstrukturen auf einem anderen Server.
Im experimentellen Abschnitt und unter Extras befinden sich rudimentäre Codeblöcke. Diese sollten NICHT ausgeführt werden!

### VORBEREITUNGSPHASE 1 - Import Modules
Pandas und os werden für das Skript benötigt.
Die pakete können einfach über die Konsole bspw. mit pip install pandas installiert werden.
Python und pip wird dazu benötigt (Falls noch nicht installiert: Googlen wie.)

In [ ]:
import sys
import numpy as np
import pandas as pd
import os
import csv

##### Check 1 - Module importiert?

In [ ]:
if "numpy" in sys.modules and "pandas" in sys.modules and "os" in sys.modules and "csv" in sys.modules:
    print("Check 1 erfolgreich.")
else:
    print("Fehler beim Import der Module. Arbeit abbrechen und Skript prüfen.")

### VORBEREITUNGSPHASE 2 - Verzeichnispfade
Es gibt das Quellverzeichnis, in dem die zu verarbeitenden Bilder liegen. Das Zielverzeichnis ist der Pfad, wo die Ordnerbäume zum Import erstellt werden sollen und in den die umbenannten Bilder zum Schluss einsortiert werden.

Im Arbeitsverzeichnis muss die Tabelle mit dem aktuellen Stand der laufenden Nummer zu jeder Seriennummer liegen (Stand_vor_ImportX.csv) und die Konkordanztabelle, welche Seriennumer zu welcher Fundortbezeichnung gehört (Seriennummer_Titel_Konkordanz.csv). Außerdem ist die Tabelle anzugeben, in der die Metadaten der Bilder, mindestens ihre Fundplatzhierarchie und Materialangaben, enthalten sind und in der die Seriennumern eingetragen werden können bzw. die Tabelle mit den aussortierten Bildern generiert werden kann.

In [ ]:
#Hier die Verzeichnispfade eingeben, wo die umzubennenenden Bilder liegen und wo sie hin sollen
quellverzeichnis = "F:\\AAArC\\FAIR.rdm AAArC 5\\Empfang\\Importe\\P11-AAArC 4-HBI\\P11_Bildimport\\P11_Bilddateien\\Aegypten_ab_Fuhre19\\AAArC_DDB_F23-F26_DIA110263-112898_fertig"
zielverzeichnis = "F:\\2022-12-02_Testlauf\\Wurst_Fuhre_23-26"

#Hier den Verzeichnispfad des Arbeitsordners angeben.
arbeitsverzeichnis = "F:\\2022-12-02_Testlauf"

#Hier die Dateinamen der Bildtabelle mit Fundorthierarchie, des Stands und der Seriennummernkonkordanz angeben.
bildtabelle = "Topo_Konkordanz_F23-F26.csv"
stand = "Stand_vor_Fuhre_23-26.csv"
seriennummernkonkordanz = "Seriennummer_Titel_Konkordanz.csv"

##### Check 2 - Pfade korrekt angegeben?

In [ ]:
if os.path.exists(quellverzeichnis):
    print("Quellverzeichnispfad OK.")
else:
    print("Quellverzeichnis nicht gefunden! Schreibweise oder Anzahl der Slashes überprüfen!")
if os.path.exists(zielverzeichnis):
    print("Zielverzeichnispfad OK.")
else:
    print("Zielverzeichnis nicht gefunden! Schreibweise oder Anzahl der Slashes überprüfen!")
if os.path.exists(arbeitsverzeichnis):
    print("Arbeitsverzeichnispfad OK.")
else:
    print("Arbeitsverzeichnis nicht gefunden! Schreibweise oder Anzahl der Slashes überprüfen!")
if os.path.exists(f"{arbeitsverzeichnis}\\{bildtabelle}"):
    print("Bildtabelle OK.")
else:
    print("Bildtabelle nicht gefunden! Schreibweise oder Anzahl der Slashes überprüfen!")
if os.path.exists(f"{arbeitsverzeichnis}\\{stand}"):
    print("Stand OK.")
else:
    print("Tabelle mit Stand nicht gefunden! Schreibweise oder Anzahl der Slashes überprüfen!")
if os.path.exists(f"{arbeitsverzeichnis}\\{seriennummernkonkordanz}"):
    print("Seriennummernkonkordanz OK.")
else:
    print("Seriennummernkonkordanz nicht gefunden! Schreibweise oder Anzahl der Slashes überprüfen!")

### TEIL 1 - Seriennummernzuordnung
#### 1.1 Seriennummern-Titel-Konkordanz einlesen

In [ ]:
df_konkordanz = pd.read_csv(f"{arbeitsverzeichnis}\\{seriennummernkonkordanz}", sep=",", dtype="string")
alle_seriennummern = list(df_konkordanz["Topographien"])
alle_titel = list(df_konkordanz["title"])
seriennummern_titel_konkordanz_dict = dict(zip(alle_titel, alle_seriennummern))
print(seriennummern_titel_konkordanz_dict)

#### 1.2 CSV mit Fundstellen einlesen und Spalte für Arachne-Snr hinzufügen

In [ ]:
df_import = pd.read_csv(f"{arbeitsverzeichnis}\\{bildtabelle}", sep=",", dtype="string", encoding="utf-8")
df_import["Arachne-Snr"]="ka"
df_import = df_import.fillna("ka")

In [ ]:
print(df_import)

#### 1.3 Noch zu importierende Seriennummern in separater CSV sammeln

In [ ]:
df_artefakte = pd.DataFrame()
df_fehler = pd.DataFrame()
print(df_artefakte)
print(df_fehler)

#### 1.4 Suchen und einfügen der benötigen Seriennummer bzw. aussortieren von Datensätzen mit Materialangabe und fehlerhaften Zuordnungen

In [ ]:
zeile = -1
for dia in df_import.iterrows():
    zeile = zeile +1
    material = df_import.at[zeile, "Material"]
    fundort1 = df_import.at[zeile, "Fundort1"]
    fundort2 = df_import.at[zeile, "Fundort2"]
    fundlandschaft = df_import.at[zeile, "Fundlandschaft"]
    fundland = df_import.at[zeile, "Fundland"]
    if material != "ka":
        print("Neue Seriennummer benötigt!")
        df_import.loc[zeile,"Arachne-Snr"] = "Neuen Datensatz anlegen"
        df_artefakte = df_artefakte.append(df_import.iloc[[zeile]])
    elif fundort1 == "ka" and fundort2 != "ka":
        if fundort2 not in seriennummern_titel_konkordanz_dict:
            print("Unbekannte Fundplatzzuordnung!")
            df_import.loc[zeile,"Arachne-Snr"] = "Unbekannte Fundplatzzuordnung!"
            df_fehler = df_fehler.append(df_import.iloc[[zeile]])
        else:
            print("Nehme Seriennummer von Fundort2...")
            arachnesnr = seriennummern_titel_konkordanz_dict[fundort2]
            df_import.loc[zeile,"Arachne-Snr"] = arachnesnr
    elif fundort1 != "ka":
        if fundort1 not in seriennummern_titel_konkordanz_dict:
            print("Unbekannte Fundplatzzuordnung!")
            df_import.loc[zeile,"Arachne-Snr"] = "Unbekannte Fundplatzzuordnung!"
            df_fehler = df_fehler.append(df_import.iloc[[zeile]])
        else:
            print("Nehme Seriennummer von Fundort1...")
            arachnesnr = seriennummern_titel_konkordanz_dict[fundort1]
            df_import.loc[zeile,"Arachne-Snr"] = arachnesnr
    elif fundort1 == "ka" and fundort2 == "ka" and fundlandschaft != "ka":
        if fundlandschaft not in seriennummern_titel_konkordanz_dict:
            print("Unbekannte Fundplatzzuordnung!")
            df_import.loc[zeile,"Arachne-Snr"] = "Unbekannte Fundplatzzuordnung!"
            df_fehler = df_fehler.append(df_import.iloc[[zeile]])
        else:
            print("Nehme Seriennummer von Fundlandschaft...")
            arachnesnr = seriennummern_titel_konkordanz_dict[fundlandschaft]
            df_import.loc[zeile,"Arachne-Snr"] = arachnesnr
    elif fundlandschaft == "ka" and fundland != "ka":
        if fundland not in seriennummern_titel_konkordanz_dict:
            print("Unbekannte Fundplatzzuordnung!")
            df_import.loc[zeile,"Arachne-Snr"] = "Unbekannte Fundplatzzuordnung!"
            df_fehler = df_fehler.append(df_import.iloc[[zeile]])
        else:
            print("Nehme Seriennummer von Fundland...")
            arachnesnr = seriennummern_titel_konkordanz_dict[fundland]
            df_import.loc[zeile,"Arachne-Snr"] = arachnesnr
    else: 
        print("Unbekannte Fundplatzzuordnung!")
        df_import.loc[zeile,"Arachne-Snr"] = "Unbekannte Fundplatzzuordnung!"
        df_fehler = df_fehler.append(df_import.iloc[[zeile]])

#### 1.5 Test, ob letzte Zeile in der Improttabelle korrekt ist

In [ ]:
print(df_import.at[zeile, "Arachne-Snr"])
print(df_import.iloc[[zeile]])

#### 1.6 Neue Importtabelle und Tabelle mit Aussortierten exportieren, sofern Datensätze aussortiert wurden.

In [ ]:
if df_fehler.empty:
    print("Seriennummernzuordnung in Ordnung!")
else:
    print("Es müssen Seriennummern nachgetragen werden oder Fundplatzzuordnungen sind fehlerhaft! Siehe fehler.csv!")
    df_fehler.to_csv(f"{arbeitsverzeichnis}//fehler.csv", sep=";", encoding="utf-8")
    
if df_artefakte.empty:
    print("Es sind keine weiteren Artefakte nachzutragen!")
else:
    print("Es müssen Artefakte nachgetragen werden! Siehe Artefaktliste.csv!")
    df_artefakte.to_csv(f"{arbeitsverzeichnis}//artefaktliste.csv", sep=";", encoding="utf-8")
    
    
    df_import.to_csv(f"{arbeitsverzeichnis}//df_import.csv", sep=";", encoding="utf-8")

In der Tabelle AussortierteBilder.csv befinden sich jetzt alle Bilder mit Materialangaben oder Fehlern bei der Seriennummern-Zuordnung.
Alle Fehler sollten behebt werden und der Vorgang wiederholt werden, bis keine Fehlermeldungen mehr in der Tabelle vorhanden sind.
Dann kann die Tabelle df_import.csv für den weiteren Prozess verwednet werden bzw. die Tabelle AussortierteBilder.csv zum Anlegen der Inventarobjekte.

ACHTUNG! Die Datei df_import.csv enthält alle Bildmetadaten - auch die der aussortierten. Wenn ein Umbennen und Verschieben durchgeführt werden soll, müssen die Datensätze ohne Seriennummer manuell aus df_import.csv entfernt werden. Ansonsten werden die Meldungen "Unbekannte Fundplatzzuordnung!" oder "Neue Seriennummer benötigt!" als Seriennummern in den Bestand aufgenommen.

### TEIL 2 - Umbenennen und Verschieben der Bilder
#### 2.0 Modus einstellen
Sollen mit artefaktliste.csv (die alle nachgetragenen Seriennummern enthalten muss!) Artefaktbilder umbenannt und verschoben werden, ist hier "True" anzugeben. Anderfalls - für Topographien - "False".

In [ ]:
artefaktimport = False

#### 2.1 Variablen definieren
Alte und neue Bildnamen werden in ein dictionary überführt. Hier ist die entsprechende CSV anzugeben.

In [ ]:
col_list = ["Bildnamen","Arachne-Snr"]

if artefaktimport == False:
    df = pd.read_csv(f"{arbeitsverzeichnis}//df_import.csv", sep=";", usecols=col_list, encoding="utf-8")
else:
    df = pd.read_csv(f"{arbeitsverzeichnis}//artefaktliste.csv", sep=";", usecols=col_list, encoding="utf-8")
    
bildnamen = list(df["Bildnamen"])
#neuenamen = list(df["NeuerBildname"])
seriennummern = list(df["Arachne-Snr"])
srndict = dict(zip(bildnamen, seriennummern))
print(srndict)
print(seriennummern)
#Check 4 - renamedict korrekt?
#Check 5 - seriennummern korrekt?

#### 2.2 Aktuellen Stand der laufenden Nummer einlesen
Stand der laufenden Nummern einlesen und Dictionary mit aktuellem Stand der laufenden Nummer erzeugen.

In [ ]:
col_list = ["Seriennummer","StandlfdNr"]
df = pd.read_csv(f"{arbeitsverzeichnis}\\{stand}", sep=",", usecols=col_list, encoding = "utf-8")
alleseriennummern =list(df["Seriennummer"])
standlfdnr = list(df["StandlfdNr"])
standdict = dict(zip(alleseriennummern, standlfdnr))
print(standdict)

#### 2.3 Renamedict erzeugen:
Das "Renamedict" ist eine Konkordanz zwischen altem Bildnamen und neuem Bildnamen.

In [ ]:
renamedict = {}
for bild in bildnamen:
    alter_bildname = bild.strip(".tif")
    seriennummer = srndict[bild]
    if seriennummer in standdict:
        newcount = (standdict.get(seriennummer))+ 1
        standdict.update({seriennummer:newcount})
        if len(str(newcount))==1:
            newname = f"{alter_bildname}_{seriennummer},00{newcount}.tif"
        elif len(str(newcount))==2:
            newname = f"{alter_bildname}_{seriennummer},0{newcount}.tif"
        else:
            newname = f"{alter_bildname}_{seriennummer},{newcount}.tif"
    else:
        standdict[seriennummer]=3
        startcount=3
        newname = f"{alter_bildname}_{seriennummer},00{startcount}.tif"
        print(f"Neue Seriennummer {seriennummer} im aktuellen Stand angelegt!")
    renamedict[bild]=newname
#print zu Kontrolle
print(standdict)
print(renamedict)

#### 2.4 Superwichtig!
Dictionary mit neu aufgezähltem Stand der laufenden Nummern in .csv speichern.

**Unbedingt speichern und beim nächsten Mal für den Stand der laufenden Nummer verwenden!**

In [ ]:
with open(f"{arbeitsverzeichnis}\\NeuerStand.csv", "w") as f:
    fieldnames = ["Seriennummer","StandlfdNr"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for key in standdict.keys():
        f.write("%s, %s\n" % (key, standdict[key]))

#### 2.5 Dateifinder aktivieren
Diese Funktion durchsucht den Verzeichnispfad.

In [ ]:
def find_all(name, path):
    result = []
    for root, dirs, files in os.walk(path):
        if name in files:
            result.append(os.path.join(root))
        #else: print("Name is invalid!")
    return result
#Check 6 - find_all aktiviert?

#### 2.6 Umbenennen

In [ ]:
#Diese Variablen sind für die Anzeige des Umbennenungsfortschritts in Prozent
x=0
gesamtzahl = len(bildnamen)

#Dieser Loop bennent die Bilder um
for bildnamen in renamedict:
    newname = renamedict.get(bildnamen)
    pfad_list = find_all(bildnamen, quellverzeichnis)
    pfad = "".join(pfad_list)
    src =f"{pfad}/{bildnamen}"
    dst =f"{pfad}/{newname}"
    if os.path.isfile(src):
        os.rename(src, dst)
        print (f"{bildnamen} wurde erfolgreich umbenannt.")
        x = x+1
    else:
        print (f" {bildnamen} existiert nicht!")
    xfortschritt = (x*100)/gesamtzahl
    xfortschritt = round(xfortschritt, 2)
    print(f"{xfortschritt}% abgeschlossen")

#### 2.7 Ordner erstellen

In [ ]:
#Verzeichnisse nach Seriennummern erstellen
for nummer in seriennummern:
    newfolder = f"{zielverzeichnis}/{nummer}"
    datenbankfertig = f"{zielverzeichnis}/{nummer}/datenbankfertig"
    arbeitskopie = f"{zielverzeichnis}/{nummer}/datenbankfertig/optimale Arbeitskopie"
    druckfertig =  f"{zielverzeichnis}/{nummer}/druckfertig"
    version =  f"{zielverzeichnis}/{nummer}/druckfertig/druckfertig 2. Version"
    rohscans =  f"{zielverzeichnis}/{nummer}/Rohscans"
    raws =  f"{zielverzeichnis}/{nummer}/Rohscans/Rohscans-RAWs"
    sonstig =  f"{zielverzeichnis}/{nummer}/Rohscans/Rohscans-sonstig"
    if not os.path.exists(newfolder):
        os.makedirs(newfolder)
    if not os.path.exists(datenbankfertig):
        os.makedirs(datenbankfertig)
    if not os.path.exists(druckfertig):
        os.makedirs(druckfertig)
    if not os.path.exists(rohscans):
        os.makedirs(rohscans)
    if not os.path.exists(arbeitskopie):
        os.makedirs(arbeitskopie)
    if not os.path.exists(version):
        os.makedirs(version)
    if not os.path.exists(raws):
        os.makedirs(raws)
    if not os.path.exists(sonstig):
        os.makedirs(sonstig)

#### 2.8 Bilder in Ordner verschieben

In [ ]:
#Alle Dateien finden und verschieben
import shutil

#Diese Variablen sind für die Anzeige des Verschiebungsfortschritts in Prozent
y=0
gesamtzahl = len(bildnamen)

for bild in renamedict.values():
    pfad_list = find_all(bild, quellverzeichnis)
    pfad = "".join(pfad_list)
    source = f"{pfad}/{bild}"
    if os.path.isfile(source):
        direction1 = bild.split("_")[1]
        direction2 = direction1.split(",")[0]
        destination = f"{zielverzeichnis}/{direction2}/Rohscans"
        test = f"{destination}/{bild}"
        if os.path.isfile(test):
            print(f"Das Bild {bild} wurde bereits verschoben!")
        else:
            shutil.move(source, destination)
            print (f"{bild} wurde erfolgreich verschoben.")
            y = y+1
    else:
        print (f" {bild} existiert nicht!")
    yfortschritt = (y*100)/gesamtzahl
    yfortschritt = round(yfortschritt, 2)
    print(f"{yfortschritt}% abgeschlossen")

### 3. Extras und experimenteller Bereich
Hier bitte nichts ausführen ohne genau zu wissen, was passiert!

In [ ]:
#EXTRA 1 - Seriennummer und laufende Nummer umstellen

#Sollte sich wieder einmal das Schema ändern, kann mit dieser Funktion nachträglich Sereinnummer und laufende Nummer
#bei bereits fertig umbenannten Bildern getauscht werden. Wichtig zu beachten:
#1. Reihung der parts
#2. Verzeichnisangabe für die find_all-Funktion

x=0
gesamtzahl = len(neuenamen)
print(neuenamen)

for bild in neuenamen:
    part1 = bild.split("_")[0]
    part2 = bild.split("_")[1]
    part3 = part2.split(".")[0]
    newname = f"{part3}_{part1}.tif"
    repairdict = {}
    repairdict[bild]=newname
    pfad_list = find_all(bild, zielverzeichnis)
    pfad = "".join(pfad_list)
    src =f"{pfad}/{bild}"
    dst =f"{pfad}/{newname}"
    if os.path.isfile(src):
        os.rename(src, dst)
        print (f"{bild} wurde erfolgreich umbenannt.")
        x = x+1
    else:
        print (f" {bild} existiert nicht!")
        x = x+1
    fortschritt = (x*100)/gesamtzahl
    fortschritt = round(fortschritt, 2)
    print(f"{fortschritt}% abgeschlossen")

In [ ]:
#Experimenteller Bereich ab hier:

In [ ]:
#Zählen der laufenden Nummer
testbild = "DIA12984-234-SFB234_23452,34"
#wenn ein Bild umbenannt wird...
cutdianame = testbild.split("_")[1]
cutnr = cutdianame.split(",")[0]
cutnr=int(cutnr)
print(cutnr)
newcount = (standdict.get(cutnr))+ 1
print(newcount)
standdict.update({cutnr:newcount})
print(standdict)


In [ ]:
#Keys in string
for key in standdict:
    newkey = str(key)
    print(newkey)
print(standdict)

In [ ]:
a = {"hello":1234,"test":23453,"david":2342345}
print(a.get("hello"))

In [ ]:
#Alle .tifs in einem Zielverzeichnis finden und in eine Liste packen
import pathlib
fileExt = r"**\*.tif"
torepair = list(str(_) for _ in pathlib.Path(zielverzeichnis).glob(fileExt))
print(torepair)

In [ ]:
#Teile der Bidnamen in .csv listen
import csv
for bild in torepair:
    part0 = bild.split("\\")[-1]
    part1 = part0.split("_")[0]
    part2 = part0.split("_")[1]
    csvrow = []
    csvrow.append(part1)
    csvrow.append(part0)
    csvrow.append(part2)
    print(csvrow)
    f = open("D:\\Testzielverzeichnis\\newnames.csv", "w", encoding="UTF8",newline = "")
    writer = csv.writer(f, delimiter=";")
    writer.writerow(csvrow)
    f.close()

In [ ]:
for bild in torepair:
    part0 = bild.split("\\")[-1]
    part1 = part0.split("_")[0]
    part2 = part0.split("_")[1]
    part3 = part2.split(".")[0]
    newname = f"{part3}_{part1}.tif"
    print(repr(bild))
    print(part0)
    print(part1)
    print(part2)
    print(part3)
    #print(newname)
    renamedict2 = {}
    renamedict2[part0]=newname
    print(renamedict2)

In [ ]:
test = "2323453,009_testbild.tif"
part1 = test.split("_")[0]
part2 = test.split("_")[1]
part3 = part2.split(".")[0]
newname = f"{part3}_{part1}.tif"
print(test)
print(newname)
renamedict2 = {}
renamedict2[test]=newname
print(renamedict2)

In [ ]:
    newname = renamedict.get(bildnamen)
    pfad_list = find_all(bildnamen, quellverzeichnis)
    pfad = "".join(pfad_list)
    src =f"{pfad}/{bildnamen}"
    dst =f"{pfad}/{newname}"
    if os.path.isfile(src):
        os.rename(src, dst)
        print (f"{bildnamen} wurde erfolgreich umbenannt.")
        x = x+1
    else:
        print (f" {bildnamen} existiert nicht!")
    fortschritt = (x*100)/gesamtzahl
    fortschritt = round(fortschritt, 2)
    print(f"{fortschritt}% abgeschlossen")

In [ ]:
a = "Bildname_Seriennummer,234"
snr1 = a.split("_")[1]
snr2 = snr1.split(",")[0]
print(snr2)

In [ ]:
class RenamePictures:
    def __init__(self, bildname, neuername, pfad)
    self.bildname = bildname
    self.neuername = neuername
    self.pfad = pfad
    
    def __renamedict(self):
        col_list = ["Bildname", "NeuerBildname"]
        

In [ ]:
string = "128471924_hallo"
print(string.split("_")[0])

In [ ]:
print(neuenamen)